In [ ]:
import os
import cv2
import numpy as np
from keras.utils import image_dataset_from_directory
import tensorflow as tf
from deepface import DeepFace
from utils import preprocess_image


In [6]:
tf.config.run_functions_eagerly(True)

In [4]:
train_ds = image_dataset_from_directory(
    "train_aligned",
    labels="inferred",
    label_mode="int",
    image_size=(152, 152),
)
val_ds = image_dataset_from_directory(
    "val_aligned",
    labels="inferred",
    label_mode="int",
    image_size=(152, 152),
)
test_ds = image_dataset_from_directory(
    "test_aligned",
    labels="inferred",
    label_mode="int",
    image_size=(152, 152),
)

deployment_ds = image_dataset_from_directory(
    "deployment_aligned",
    labels="inferred",
    label_mode="int",
    image_size=(152, 152),
)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
    

Found 12866 files belonging to 288 classes.
Found 3046 files belonging to 63 classes.
Found 3099 files belonging to 63 classes.
Found 3892 files belonging to 80 classes.


In [5]:
model = DeepFace.build_model("DeepFace")
model.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.model.fit(train_ds, validation_data=val_ds, epochs=14)

In [10]:
test_database = {}
deployment_database = {}

In [26]:
def get_average_embedding(path):
    total_embeddings = None
    for img in os.listdir(path):
        image = cv2.imread(os.path.join(path, img))
        image = np.expand_dims(image, axis=0)
        try:
            detected_face_embedding = model.find_embeddings(image)
        except:
            continue
        if total_embeddings is None:
            total_embeddings = detected_face_embedding
        else:
            total_embeddings = np.vstack((total_embeddings, detected_face_embedding))
    return np.mean(total_embeddings, axis=0)


def build_database(paths, dataset):
    for person in paths:
        person_name = person.split("/")[-1]
        if person_name not in dataset:
            dataset[person_name] = get_average_embedding(person)
    return dataset


In [27]:
test_paths = os.listdir("test_aligned")
test_paths = [os.path.join("test_aligned", path) for path in test_paths if not path.startswith(".")]
test_dataset = build_database(test_paths, test_database)

In [ ]:
deployment_paths = os.listdir("deployment_aligned")
deployment_paths = [os.path.join("deployment_aligned", path) for path in deployment_paths if not path.startswith(".")]
deployment_dataset = build_database(deployment_paths, deployment_database)

In [134]:
from tqdm import tqdm

correct = 0
total = 0

for images, labels in tqdm(test_ds):
    for img, label in tqdm(zip(images, labels)):
        img = tf.expand_dims(img, axis=0)
        try:
            detected_face_embedding = model.find_embeddings(img)
        except:
            continue
        distances = []
        for person in test_dataset:
            distances.append(np.linalg.norm(detected_face_embedding - test_dataset[person]))
        if np.argmin(distances) == label:
            correct += 1
        total += 1
    
correct/total
    

  0%|          | 0/97 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  2.95it/s]
2it [00:00,  2.88it/s]
3it [00:01,  2.81it/s]
4it [00:01,  2.87it/s]
5it [00:01,  2.92it/s]
6it [00:02,  2.95it/s]
7it [00:02,  2.96it/s]
8it [00:02,  2.98it/s]
9it [00:03,  2.99it/s]
10it [00:03,  2.94it/s]
11it [00:03,  2.96it/s]
12it [00:04,  2.97it/s]
13it [00:04,  2.98it/s]
14it [00:04,  2.99it/s]
15it [00:05,  2.99it/s]
16it [00:05,  2.97it/s]
17it [00:05,  2.97it/s]
18it [00:06,  2.99it/s]
19it [00:06,  3.00it/s]
20it [00:06,  3.01it/s]
21it [00:07,  3.03it/s]
22it [00:07,  3.03it/s]
23it [00:07,  3.03it/s]
24it [00:08,  3.02it/s]
25it [00:08,  3.02it/s]
26it [00:08,  3.01it/s]
27it [00:09,  3.02it/s]
28it [00:09,  3.02it/s]
29it [00:09,  3.02it/s]
30it [00:10,  3.02it/s]
31it [00:10,  3.03it/s]
32it [00:10,  2.99it/s]
  1%|          | 1/97 [00:10<17:11, 10.75s/it]
0it [00:00, ?it/s]
1it [00:00,  3.01it/s]
2it [00:00,  3.03it/s]
3it [00:00,  3.02it/s]
4it [00:01,  3.01it/s]
5it [00:01,  3.02it/s]
6i

0.026460148434979024

In [ ]:
def infer_person(path, dataset):
    image = cv2.imread(path)
    preprocessed = preprocess_image(image)
    detected_face_embedding = model.find_embeddings(preprocessed)
    distances = []
    for person in dataset:
        distances.append(np.linalg.norm(detected_face_embedding - dataset[person]))
    return list(dataset.keys())[np.argmin(distances)]

In [3]:
def save_database(dataset, path):
    for person in dataset:
        np.save(os.path.join(path, person.split("/")[-1]), dataset[person])
        
def load_database(path):
    dataset = {}
    for person in os.listdir(path):
        person_name = person.split(".")[0]
        dataset[person_name] = np.load(os.path.join(path, person))
    return dataset

def add_person_to_database(path, dataset_path):
    dataset = load_database(dataset_path)
    dataset[path] = get_average_embedding(path)
    save_database(dataset, dataset_path)
    return dataset

In [ ]:
model.model.save("deepface_model.h5")

In [7]:
model.model = tf.keras.models.load_model("deepface_model.h5")